# Image Classification

# Apples and Bananas

The following model classifies two different fruit categories - Apples and Bananas. Please feel free to change the categories and classify the items/classes of your choice.

Please note that the below model works for classifying only two categories. If you want to classify more than two classes, use one hot encoding for the labels and use the loss function as "categorical_cross_entropy" and update the number of nodes in the final Dense layer equal to number of classes you want to classify.

# Import the required libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [2]:
#This is a path to your dataset directory.
DATADIR = "C:/Users/Idea/Desktop/Fruits/datasets"

#Names of the categories the model predicts
CATEGORIES = ["Apples", "Bananas"]

#Train the model by inputting the images of size 50X50. It is a choice.
IMG_SIZE = 50

# Create training data

In [3]:
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category) 
        for img in tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE) 
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
                training_data.append([new_array, class_num]) 
            except Exception as e:
                pass

create_training_data()

100%|████████████████████████████████████████████████████████████████████████████████| 218/218 [00:04<00:00, 48.64it/s]


# Shuffle the training data

In [4]:
import random

random.shuffle(training_data)

# Generate attributes/features and decisions/labels

In [5]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

# Convert the features and labels into numpy arrays

In [6]:
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

# Normalising the features

In [7]:
X = X/255.0

# Build your Model

In [9]:
def create_model():

    model = Sequential()
    
    #Input Layer
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(50,50,1)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.3))
    
    #Layer
    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.3),
    
    #Flattening before the fully-connected network
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    
    #Final Output Layer
    model.add(Dense(2, activation='softmax'))
  
   #Compile the model
    model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
         optimizer=tf.keras.optimizers.Adam(),
         metrics=['accuracy'])
      
    return model


model = create_model()


model.fit(X, y,
          batch_size=32,
          epochs=5,
          validation_split=0.1)

Train on 405 samples, validate on 45 samples
Epoch 1/5
405/405 [==============================] - ETA: 1:17 - loss: 0.6802 - accuracy: 0.62 - ETA: 38s - loss: 1.3041 - accuracy: 0.5781 - ETA: 24s - loss: 1.1620 - accuracy: 0.552 - ETA: 17s - loss: 1.1842 - accuracy: 0.554 - ETA: 12s - loss: 1.0878 - accuracy: 0.581 - ETA: 9s - loss: 1.0339 - accuracy: 0.588 - ETA: 7s - loss: 1.0030 - accuracy: 0.59 - ETA: 5s - loss: 0.9908 - accuracy: 0.59 - ETA: 3s - loss: 0.9535 - accuracy: 0.59 - ETA: 2s - loss: 0.9166 - accuracy: 0.59 - ETA: 1s - loss: 0.8907 - accuracy: 0.59 - ETA: 0s - loss: 0.8753 - accuracy: 0.60 - 10s 25ms/sample - loss: 0.8666 - accuracy: 0.6099 - val_loss: 0.4700 - val_accuracy: 0.7778
Epoch 2/5
405/405 [==============================] - ETA: 1s - loss: 0.5234 - accuracy: 0.71 - ETA: 1s - loss: 0.4895 - accuracy: 0.75 - ETA: 1s - loss: 0.5129 - accuracy: 0.72 - ETA: 0s - loss: 0.5375 - accuracy: 0.71 - ETA: 0s - loss: 0.5419 - accuracy: 0.70 - ETA: 0s - loss: 0.5250 - accura

# Save your trained model

In [10]:
keras_file = "fruits.h5"
tf.keras.models.save_model(model, keras_file)

# Convert your saved .h5 model file to tflite file

In [11]:
model = tf.keras.models.load_model("fruits.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("fruits.tflite", "wb").write(tflite_model)

10244700